In [163]:
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import glob 
import cv2
import matplotlib.pyplot as plt
import numpy as np
import math
import random
import os
import shutil

In [164]:
def move_files_to_folder(folder_path):
    # Iterate through folders 1, 2, 3, and 4
    for folder_name in ['Dat1', 'Dat2', 'Duong1', 'Duong2']:
        source_folder = os.path.join(folder_path, folder_name)
        # Check if the folder exists
        if os.path.exists(source_folder):
            # Move files from the source folder to the main folder
            for filename in os.listdir(source_folder):
                source_file = os.path.join(source_folder, filename)
                destination_file = os.path.join(folder_path, filename)
                # Check if the file already exists in the destination folder
                if not os.path.exists(destination_file):
                    shutil.move(source_file, destination_file)
                else:
                    print(f"File '{filename}' already exists in the destination folder.")
            shutil.rmtree(source_folder)
        else:
            print(f"Folder '{folder_name}' does not exist in '{folder_path}'.")

In [165]:
folder_path = "/Users/khuenguyen/Desktop/Vehicle_Detection/vehicle_detection_models/vehicle_dataset"
move_files_to_folder(folder_path)

File 'classes.txt' already exists in the destination folder.
File 'classes.txt' already exists in the destination folder.
File 'classes.txt' already exists in the destination folder.


In [166]:
images = []
images_path = glob.glob("vehicle_dataset/*.jpg")

In [167]:
new_image_path = []

In [168]:
labels = [0,1,2,3,4,5]
labels_dict = {0:0,1:1,2:2,3:3,4:0,5:2}
new_labels = [0,1,2,3]

In [169]:
for img_path in images_path:
    img_name = img_path[:-3]
    label_path = img_name + 'txt'
    xys = []
    if not os.path.isfile(label_path):
        os.remove(img_path)
        continue
    with open(label_path,'r') as file:
        for line in file:
            label,x,y,w,h = [float(x) for x in line.split()]
            if(int(label) in labels):
                label = labels_dict[int(label)]
                xys.append((label,x,y,w,h))
            else:
                print("UNKNOWN LABEL FOUND!")
                break

    with open(label_path,'w') as file:
    
        for label,x,y,w,h in xys:
            if(int(label) in new_labels):
                file.write(f"{int(label)} {x} {y} {w} {h}\n")
            else:
                print(f"Wrong labels found for {label_path}")
    new_image_path.append(img_path)


    

In [170]:
labels = new_labels

In [171]:
for img_path in new_image_path:
    img_name = img_path[:-3]
    label_path = img_name + 'txt'
    if not os.path.isfile(label_path):
        print(f"No label for {img_path}")
        os.remove(img_path)
        images_path.remove(img_path)
        continue
    with open(label_path,'r') as file:
        for line in file:
            label,x,y,w,h = [float(x) for x in line.split()]
            if(int(label) in labels):
                pass
            else:
                print("UNKNOWN LABEL FOUND!")
                print(label_path)
                break

In [172]:
labels

[0, 1, 2, 3]

In [173]:
print(images_path[0])

vehicle_dataset/final.5493094c-e1b9-11ee-ae34-a6d57e25c571.jpg


In [174]:
def yolobbox2bbox(x,y,w,h,dw,dh):
    x1, y1 = (x-w/2)*dw, (y-h/2)*dh
    x2, y2 = (x+w/2)*dw, (y+h/2)*dh
    return ((x1), (y1), (x2), (y2))

In [175]:
def save_file(img_name,img):
    cv2.imwrite(img_name, img)
def convert(size, box):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[1])/2.0
    y = (box[2] + box[3])/2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h) 
def pascal_voc_to_yolo(x1, y1, x2, y2, image_w, image_h):
    return [((x2 + x1)/(2*image_w)), ((y2 + y1)/(2*image_h)), ((x2 - x1)/image_w), ((y2 - y1)/image_h)]

In [176]:
augmentation = iaa.Sequential([
        iaa.OneOf([ ## rotate
            iaa.Affine(rotate=0),
            iaa.Affine(rotate=90)
        ]),

        iaa.Fliplr(0.5),
        iaa.Flipud(0.2),

        iaa.OneOf([ # drop out augmentation
            iaa.CoarseDropout((0.0, 0.05), size_percent=(0.02, 0.25)),
            ]),

        iaa.OneOf([ ## weather augmentation
            iaa.Rain(speed=(0.3, 0.5)),
        ]),  

        iaa.OneOf([ ## brightness or contrast
            iaa.Multiply((0.8, 1.0)),
            iaa.contrast.LinearContrast((0.9, 1.1)),
        ]),

        iaa.OneOf([ ## blur or sharpen
            iaa.GaussianBlur(sigma=(0.0, 0.1)),
            iaa.Sharpen(alpha=(0.0, 0.1)),
        ])
    ],
    # do all of the above augmentations in random order
    random_order=True
)

In [177]:
for img_path in new_image_path:
    img_name = img_path[:-3]
    label_path = img_name + 'txt'
    if not os.path.isfile(label_path):
        os.remove(img_path)
        images_path.remove(img_path)
        continue
    delete = False
    with open(label_path,'r') as file:
        xys = []
        for line in file:
            label,x,y,w,h = [float(x) for x in line.split()]
            if(int(label) in labels):
                pass
            else:
                print("UNKNOWN LABEL FOUND!")
                print(label)
                print(img_path)
                delete = True
                break
    if delete:
        print(img_path)
        os.remove(img_path)
        os.remove(label_path)
        images_path.remove(img_path)

In [178]:
aug_num = 0
for img_path in new_image_path:
    img = cv2.imread(img_path)
    images.append(img)
    img_name = img_path[:-3]
    label_path = img_name + 'txt'
    dh, dw, _ = img.shape
    if not os.path.isfile(label_path):
        os.remove(img_path)
        images_path.remove(img_path)
        continue
    with open(label_path,'r') as file:
        xys = []
        for line in file:
            label,x,y,w,h = [float(x) for x in line.split()]
            if(int(label) in labels):
                x1,y1,x2,y2 = yolobbox2bbox(x,y,w,h,dw,dh)
                xys.append((x1,y1,x2,y2,label))
            else:
                print("UNKNOWN LABEL FOUND!")
                print(label)
                print(img_path)
                break
        bbs = BoundingBoxesOnImage([BoundingBox(x1,y1,x2,y2,label) for x1,y1,x2,y2,label in xys],shape=img.shape)
        aug_img, aug_bbs = augmentation(image = img,bounding_boxes=bbs)
        save_file(f"vehicle_dataset/augmented_vehicle_{aug_num}.jpg",aug_img)
        dh, dw, _ = aug_img.shape
        with open(f"vehicle_dataset/augmented_vehicle_{aug_num}.txt",'w') as f:
            for box in aug_bbs.bounding_boxes:
                x,y,w,h = pascal_voc_to_yolo(box.x1,box.y1,box.x2,box.y2,dw,dh)
                if(x<0): x = 0
                if(x>1): x = 1
                if(y<0): y = 0
                if(y>1): y = 1
                f.write(f"{int(label)} {x} {y} {w} {h}\n")
        aug_num+=1